In [2]:
import numpy as np
import pandas as pd

from keras import layers
from keras.applications import DenseNet121
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import Callback, ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.optimizers import Adam

Using TensorFlow backend.


In [3]:
train = pd.read_csv('csv/train_processed.csv')

In [4]:
train.head()

,Image,any,epidural,intraparenchymal,intraventricular,subarachnoid,subdural
0,ID_000039fa0.png,0,0,0,0,0,0
1,ID_00005679d.png,0,0,0,0,0,0
2,ID_00008ce3c.png,0,0,0,0,0,0
3,ID_0000950d7.png,0,0,0,0,0,0
4,ID_0000aee4b.png,0,0,0,0,0,0


In [6]:
BATCH_SIZE = 64

def create_datagen():
    return ImageDataGenerator(validation_split=0.2)

def create_test_gen():
    return ImageDataGenerator().flow_from_dataframe(
        test_df,
        directory='data/test_stage1_png/',
        x_col='Image',
        class_mode=None,
        target_size=(224, 224),
        batch_size=BATCH_SIZE,
        shuffle=False
    )

def create_flow(datagen, subset):
    return datagen.flow_from_dataframe(
        train, 
        directory='data/train_stage1_png/',
        x_col='Image', 
        y_col=['any', 'epidural', 'intraparenchymal', 
               'intraventricular', 'subarachnoid', 'subdural'],
        class_mode='other',
        target_size=(224, 224),
        batch_size=BATCH_SIZE,
        subset=subset
    )

# Using original generator
data_generator = create_datagen()
train_gen = create_flow(data_generator, 'training')
val_gen = create_flow(data_generator, 'validation')
#test_gen = create_test_gen()

Found 537437 validated image filenames.
Found 134359 validated image filenames.


In [7]:
densenet = DenseNet121(
    include_top=False,
    input_shape=(224,224,3)
)

Instructions for updating:
Colocations handled automatically by placer.
29089792/29084464 [==============================] - 4s 0us/step


In [8]:
def build_model():
    model = Sequential()
    model.add(densenet)
    model.add(layers.GlobalAveragePooling2D())
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(6, activation='sigmoid'))
    
    model.compile(
        loss='binary_crossentropy',
        optimizer=Adam(lr=0.001),
        metrics=['accuracy']
    )
    
    return model

In [9]:
model = build_model()
model.summary()

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
densenet121 (Model)          (None, 7, 7, 1024)        7037504   
_________________________________________________________________
global_average_pooling2d_1 ( (None, 1024)              0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 6150      
Total params: 7,043,654
Trainable params: 6,960,006
Non-trainable params: 83,648
_________________________________________________________________


In [ ]:
checkpoint = ModelCheckpoint(
    'model.h5', 
    monitor='val_loss', 
    verbose=0, 
    save_best_only=True, 
    save_weights_only=False,
    mode='auto'
)

total_steps = 537437 / BATCH_SIZE

history = model.fit_generator(
    train_gen,
    steps_per_epoch=total_steps * 0.85,
    validation_data=val_gen,
    validation_steps=total_steps * 0.15,
    callbacks=[checkpoint],
    epochs=5
)

Instructions for updating:
Use tf.cast instead.
Epoch 1/5
